<a href="https://colab.research.google.com/github/hellomstars/Kaggle_Titanic/blob/main/titanic_%EC%95%88%EC%84%9C%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random as rnd

In [ ]:
train_df=pd.read_csv("./titanic/train.csv")
test_df=pd.read_csv("./titanic/test.csv")

In [ ]:
print(train_df.columns.values)
print(test_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']
['PassengerId' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare'
 'Cabin' 'Embarked']


In [ ]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
#변수 제거
train_data = train_df.drop(['Ticket', 'Cabin', 'PassengerId'], axis=1)
test_data = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_data, test_data]

train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [ ]:
#이름 추출
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train_data['Title'], train_data['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_data[['Title', 'Age']].groupby(['Title'], as_index=False).mean()

,Title,Age
0,Capt,70.000000
1,Col,58.000000
2,Countess,33.000000
3,Don,40.000000
4,Dr,42.000000
5,Jonkheer,38.000000
6,Lady,48.000000
7,Major,48.500000
8,Master,4.574167
9,Miss,21.845638


In [ ]:
Title_age_mean = train_data[['Title', 'Age']].groupby(['Title'], as_index=False).mean()

In [ ]:
int(Title_age_mean.loc[Title_age_mean.Title == 'Mr', 'Age'])

32

In [ ]:
#비어있는 나이 채워넣기
for dataset in combine:
    dataset.loc[ (dataset.Age.isnull()) & (dataset.Title == 'Mr'),'Age'] = int(Title_age_mean.loc[Title_age_mean.Title == 'Mr', 'Age'])
    dataset.loc[ (dataset.Age.isnull()) & (dataset.Title == 'Mrs'),'Age'] = int(Title_age_mean.loc[Title_age_mean.Title == 'Mrs', 'Age'])
    dataset.loc[ (dataset.Age.isnull()) & (dataset.Title == 'Miss'),'Age'] = int(Title_age_mean.loc[Title_age_mean.Title == 'Miss', 'Age'])
    dataset.loc[ (dataset.Age.isnull()) & (dataset.Title == 'Master'),'Age'] = int(Title_age_mean.loc[Title_age_mean.Title == 'Master', 'Age'])
    dataset.loc[ (dataset.Age.isnull()) & (dataset.Title == 'Dr'),'Age'] = int(Title_age_mean.loc[Title_age_mean.Title == 'Dr', 'Age'])

In [ ]:
train_data.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
Title       0
dtype: int64

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Etc')

train_data[['Title', 'Age']].groupby(['Title'], as_index=False).mean()

,Title,Age
0,Etc,45.391304
1,Master,4.516750
2,Miss,21.681081
3,Mr,32.283366
4,Mrs,35.682540


In [ ]:
#선착장 유실 정보 채워넣기
for dataset in combine:
    dataset.loc[ (dataset.Embarked.isnull()),'Embarked'] = 'S'
    
train_data.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
Title       0
dtype: int64

In [ ]:
#가족 수 따라서
for dataset in combine:
    dataset['FamilyNum'] = dataset['SibSp'] + dataset['Parch'] + 1

train_data[['FamilyNum', 'Survived']].groupby(['FamilyNum'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilyNum,Survived
3,4,0.724138
2,3,0.578431
1,2,0.552795
6,7,0.333333
0,1,0.303538
4,5,0.200000
5,6,0.136364
7,8,0.000000
8,11,0.000000


In [ ]:
#혼자인지 가족이 있는지 여부 확인
for dataset in combine:
    dataset['isAlone'] = 0
    dataset.loc[dataset['FamilyNum'] == 1, 'isAlone'] = 1

train_data[['isAlone', 'Survived']].groupby('isAlone', as_index = False).mean()

,isAlone,Survived
0,0,0.505650
1,1,0.303538


In [ ]:
train_set = train_data.drop(['Name', 'Parch', 'SibSp', 'FamilyNum'], axis=1)
test_set = test_data.drop(['Name', 'Parch', 'SibSp', 'FamilyNum'], axis=1)
combine = [train_set, test_set]
train_set.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,isAlone
0,0,3,male,22.0,7.2500,S,Mr,0
1,1,1,female,38.0,71.2833,C,Mrs,0
2,1,3,female,26.0,7.9250,S,Miss,1
3,1,1,female,35.0,53.1000,S,Mrs,0
4,0,3,male,35.0,8.0500,S,Mr,1


In [ ]:
#Fare 정규화

def normal(data):
    
    data = (data - data.min())/(data.max() - data.min())
    
    return data

Fare_scaled = normal(train_set['Fare'])
Fare_scaled

0      0.014151
1      0.139136
2      0.015469
3      0.103644
4      0.015713
         ...   
886    0.025374
887    0.058556
888    0.045771
889    0.058556
890    0.015127
Name: Fare, Length: 891, dtype: float64

In [ ]:
for dataset in combine:
    dataset['Fare'] = Fare_scaled

In [ ]:
train_set.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,isAlone
0,0,3,male,22.0,0.014151,S,Mr,0
1,1,1,female,38.0,0.139136,C,Mrs,0
2,1,3,female,26.0,0.015469,S,Miss,1
3,1,1,female,35.0,0.103644,S,Mrs,0
4,0,3,male,35.0,0.015713,S,Mr,1


In [ ]:
combine = [train_set, test_set]
train_set.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,isAlone
0,0,3,male,22.0,0.014151,S,Mr,0
1,1,1,female,38.0,0.139136,C,Mrs,0
2,1,3,female,26.0,0.015469,S,Miss,1
3,1,1,female,35.0,0.103644,S,Mrs,0
4,0,3,male,35.0,0.015713,S,Mr,1


In [ ]:
#수치형으로 변환

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Etc": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    dataset['Sex'] = dataset['Sex'].map( {'male': 0, 'female': 1} ).astype(int)
    dataset['Embarked'] = dataset['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

train_set.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,isAlone
0,0,3,0,22.0,0.014151,0,1,0
1,1,1,1,38.0,0.139136,1,3,0
2,1,3,1,26.0,0.015469,0,2,1
3,1,1,1,35.0,0.103644,0,3,0
4,0,3,0,35.0,0.015713,0,1,1


In [ ]:
train_set.to_csv("Titanic train 전처리_안서현1103.csv", index = True)
test_set.to_csv("Titanic test 전처리_안서현1103.csv", index = True)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# 목적 변수 제거
X_train = train_set.drop("Survived", axis=1)
#목적 변수 역할
Y_train = train_set["Survived"]
#예측 대상 데이터 셋
X_test  = test_set.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 7), (891,), (418, 7))

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

80.13

In [ ]:
coeff_df = pd.DataFrame(train_set.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
1,Sex,2.188451
5,Title,0.386843
6,isAlone,0.294263
4,Embarked,0.294248
3,Fare,0.268385
2,Age,-0.033821
0,Pclass,-1.160577


In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

74.07

In [ ]:
#KNN
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

87.99

In [ ]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

77.67

In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

74.3

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
print(acc_linear_svc)

65.77


C:\Users\hello\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

69.25

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

98.2

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

98.2

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,98.20
8,Decision Tree,98.20
1,KNN,87.99
2,Logistic Regression,80.13
4,Naive Bayes,77.67
5,Perceptron,74.30
0,Support Vector Machines,74.07
6,Stochastic Gradient Decent,69.25
7,Linear SVC,65.77


In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_set["PassengerId"],
        "Survived": Y_pred
    })

In [ ]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
submission.to_csv("Submission_SeoHyunAhn.csv", index = False)